# Learning Filter Scale by $\partial\sigma$ (in PyTorch)

In [ ]:
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

torch.set_default_dtype(torch.float64)

torch.manual_seed(1337)

## Gaussian Filter and Gradient w.r.t. $\sigma$

Make a Gaussian filter from the Gaussian distribution with mean zero and a given standard deviation $\sigma$. 
The Gaussian distribution is discretized and truncated to give a filter with finite support for efficient filtering by convolution. 
For simplicity, we calculate the unnormalized Gaussian density then normalize it by dividing by the sum, which also corrects for the missing density discarded by truncation.
The quality of the filter w.r.t. to the true Gaussian distribution is determined by the number of elements in the filter, or kernel size.
By parameterizing the kernel size in terms of the standard deviation, we can control the effect of truncation, and for instance include 95% of the true density by making the support cover two standard deviations from the mean.

In [ ]:
sigma = torch.Tensor([3.])

# determine kernel size to cover +/- 2 sigma s.t. >95% of density is included
half_size = int(max(1, torch.ceil(sigma*3)))
# always make odd kernel to keep coordinates centered
kernel_size = half_size*2 + 1
# calculate unnormalized density then normalize
x = torch.linspace(-half_size, half_size, steps=kernel_size)
filter_ = torch.exp(-x**2 / (2*sigma**2))
filter_sum = filter_.sum()
filter_norm = filter_ / filter_sum

plt.figure()
plt.title("Gaussian filter")
plt.plot(x.numpy(), filter_norm.numpy())

Differentiate the normalized filter w.r.t. standard deviation $\sigma$ through the chain and quotient rules.

In [ ]:
d_filter_sigma = filter_ * x**2 / sigma**3
d_filter_sigma_sum = d_filter_sigma.sum()
d_filter_norm_sigma = (filter_sum * d_filter_sigma - filter_ * d_filter_sigma_sum) / filter_sum**2


plt.figure()
plt.title("Gradient of filter w.r.t. $\sigma$")
plt.plot(x.numpy(), d_filter_norm_sigma.numpy())

Package up filter and gradient computation so that backward can re-use forward computation of the filter and sum.

In [ ]:
def gaussian_filter_with_grad(sigma):
    # determine kernel size to cover +/- 2 sigma s.t. >95% of density is included
    # always make odd kernel to keep coordinates centered
    half_size = int(max(1, torch.ceil(sigma*3)))
    kernel_size = half_size*2 + 1
    # calculate unnormalized density then normalize
    x = torch.linspace(-half_size, half_size, steps=kernel_size)
    filter_ = torch.exp(-x**2 / (2*sigma**2))
    filter_sum = filter_.sum()
    filter_norm = filter_ / filter_sum
    # gradient w.r.t. sigma
    d_filter_sigma = (filter_ * x**2) / sigma**3
    d_filter_sigma_sum = d_filter_sigma.sum()
    d_filter_norm_sigma = (filter_sum * d_filter_sigma - filter_ * d_filter_sigma_sum) / filter_sum**2
    return filter_norm, d_filter_norm_sigma

## Parameterizing $\sigma$ for Learning

Parametrize sigma through a learned parameter $s$ and define the gradient of the parameter. For unconstrained optimization, we define the parameter in [-inf, +inf] and map it into a valid sigma with a minimum and default of our choosing.

In [ ]:
# minimum sigma = 0.3 gives a little bit of blur, but is closer than not to a delta
MIN_SIGMA = torch.tensor([0.3])
# sigma parameter shift determines the blur at zero (to make default delta-like)
# remember, this also determines the basin for weight decay
SIGMA_PARAM_SHIFT = torch.tensor([-3.])

# the sigma parameter, s
s = torch.tensor([0.])

def sigma_from_param(s):
    # map to valid sigma by the soft max (not the softmax!)
    # https://www.johndcook.com/blog/2010/01/13/soft-maximum/
    sigma = torch.log(torch.exp(MIN_SIGMA) + torch.exp(s + SIGMA_PARAM_SHIFT))
    return sigma

def sigma_param_grad(s):
    # gradient by chain rule
    d_param = torch.exp(s + SIGMA_PARAM_SHIFT) / (torch.exp(MIN_SIGMA) + torch.exp(s + SIGMA_PARAM_SHIFT))
    return d_param


sigma = sigma_from_param(s)
filter_, _ = gaussian_filter_with_grad(sigma)
plt.figure()
plt.title("Gaussian filter for s = 0")
plt.plot(filter_.numpy())

Filter with differentiable smoothing: forward filters with a given sigma parameter and backward differentiates w.r.t. the sigma parameter.

In [ ]:
def sigma_filter_forward(x, s):
    f, d_f_sigma = gaussian_filter_with_grad(sigma_from_param(s))
    xf = F.conv1d(x.view(1, 1, -1), f.view(1, 1, -1), padding=f.size(0) // 2)  # "same" convolution
    xf = xf[0, 0]  # discard dummy dims
    return xf, d_f_sigma

def sigma_filter_backward(x, s, d_loss, d_f_sigma):
    half_width = d_f_sigma.size(0) // 2
    d_f_pad = F.pad(d_loss, (half_width, half_width))
    # n.b. we don't flip the data x b.c. convolution in pytorch is cross-correlation
    d_f = F.conv1d(d_f_pad.view(1, 1, -1), x.view(1, 1, -1))  # "valid" convolution
    d_f = d_f[0, 0]  # discard dummy dims
    d_sigma = (d_f * d_f_sigma).sum()
    d_s = d_sigma * sigma_param_grad(s)
    return d_s

## Toy Experiment: Optimize $\sigma$ to Recover Blur Kernel

To illustrate the optimization of kernel size via sigma with a toy problem, let's recover the size of a Gaussian blur from smoothed data in 1D.

1. Generate a random 1D signal and smooth it with a reference sigma.
2. Instantiate our filter with zero initialization of the sigma parameter.
3. Learn sigma by gradient descent.

In [ ]:
x = torch.randn(100)
true_sigma = torch.tensor([3.])
true_blur, _ = gaussian_filter_with_grad(true_sigma)
xf = F.conv1d(x.view(1, 1, -1), true_blur.view(1, 1, -1), padding=true_blur.size(0) // 2)[0, 0]

plt.figure(figsize=(10, 2))
plt.subplot(1, 2, 1)
plt.title('signal')
plt.plot(x.numpy())
plt.subplot(1, 2, 2)
plt.title('smoothed')
plt.plot(xf.numpy())

In [ ]:
def plot_recovery(xf, xf_hat, iter_):
    plt.figure(figsize=(5, 2))
    plt.title("Recovery iter. {}".format(iter_))
    plt.plot(xf.numpy(), 'b', label='ref.')
    plt.plot(xf_hat.numpy(), 'r', label='rec.')
    plt.legend()
    
s = torch.tensor([0.])

max_iter = 100
step_size = torch.tensor([0.1])
for iter_ in range(max_iter):
    xf_hat, d_f_sigma = sigma_filter_forward(x, s)
    # loss: squared error
    loss = 0.5 * ((xf_hat - xf)**2).sum()
    if iter_ % 10 == 0:
        print('loss ', loss.item())
    if iter_ in (0, 4, 16):
        plot_recovery(xf, xf_hat, iter_)
    d_loss = xf_hat - xf
    d_s = sigma_filter_backward(x, s, d_loss, d_f_sigma)
    # update
    s -= step_size * d_s
plot_recovery(xf, xf_hat, iter_ + 1)

sigma_hat = sigma_from_param(s)
print('\ntrue sigma {:0.2f} recovered sigma {:0.2f}'.format(true_sigma.item(), sigma_hat.item()))

Check the gradient by finite differences.

In [ ]:
eps = torch.tensor([1e-5])
for _ in range(10):
    s = torch.randn(1) * 3.
    # forward-backward
    xf_hat, d_f_sigma = sigma_filter_forward(x, s)
    loss = 0.5 * ((xf_hat - xf)**2).sum()
    d_loss = xf_hat - xf
    d_s = sigma_filter_backward(x, s, d_loss, d_f_sigma)
    
    # forward +eps
    xf_eps, _ = sigma_filter_forward(x, s + eps)
    loss_ = 0.5 * ((xf_eps - xf)**2).sum()
    d_s_check = (loss_ - loss) / eps
    #print('loss ', loss.item(), loss_.item(), (loss - loss_).item())
    #print('s ', s.item(), (s + eps).item(), (s - (s + eps)).item())
    err = torch.abs(d_s_check - d_s)
    print('analytic {: 09.5f} numerical {: 09.5f} error {:0.8f}'.format(d_s.item(), d_s_check.item(), err.item()))
    assert(err < 10*eps)